In [28]:
import pickle

d = pickle.load(open('ml_latest-small_core_10_type_hete.pkl', 'rb'))

In [29]:
d.keys()

dict_keys(['unique_uids', 'num_uids', 'unique_iids', 'num_iids', 'unique_genres', 'num_genres', 'unique_years', 'num_years', 'unique_directors', 'num_directors', 'unique_actors', 'num_actors', 'unique_writers', 'num_writers', 'unique_tids', 'num_tids', 'num_nodes', 'num_node_types', 'e2nid_dict', 'nid2e_dict', 'rating_np', 'edge_index_nps', 'test_pos_unid_inid_map', 'neg_unid_inid_map', 'edge_type_dict', 'num_edge_types', 'item_nid_occs', 'types', 'num_nodes_dict', 'type_accs'])

In [30]:
# rating_np
# edges are in d['edge_index_nps']['user2item'] and values are in d['rating_np']
# create a dictionary of user2item edges and ratings

user2item = d['edge_index_nps']['user2item']
ratings = d['rating_np']

edge_values = {(int(user2item[0][i]), int(user2item[1][i])): ratings[i] / 5 for i in range(len(user2item[0]))}


In [31]:
edge_values[(440, 1741)]

KeyError: (440, 1741)

In [ ]:
d.keys()

dict_keys(['unique_uids', 'num_uids', 'unique_iids', 'num_iids', 'unique_genres', 'num_genres', 'unique_years', 'num_years', 'unique_directors', 'num_directors', 'unique_actors', 'num_actors', 'unique_writers', 'num_writers', 'unique_tids', 'num_tids', 'num_nodes', 'num_node_types', 'e2nid_dict', 'nid2e_dict', 'rating_np', 'edge_index_nps', 'test_pos_unid_inid_map', 'neg_unid_inid_map', 'edge_type_dict', 'num_edge_types', 'item_nid_occs', 'types', 'num_nodes_dict', 'type_accs'])

In [ ]:
import numpy as np

# unique, counts = np.unique(d['edge_index_nps']['user2item'][0], return_counts=True)

In [ ]:
# np.std(counts)

In [ ]:
import pandas as pd

In [37]:
movies = pd.read_csv('../raw/raw_movies.csv', sep=';').fillna('')
ratings = pd.read_csv('../raw/raw_ratings.csv', sep=';')
tagging = pd.read_csv('../raw/raw_tagging.csv', sep=';')

In [38]:
def reindex_df_mlsmall(movies, ratings, tagging):
    """

    Args:
        movies:
        ratings:
        tagging:
        genome_tagging:
        genome_tags:

    Returns:

    """
    # Reindex uid
    unique_uids = np.sort(ratings.uid.unique()).astype(int)
    uids = np.arange(unique_uids.shape[0]).astype(int)
    raw_uid2uid = {raw_uid: uid for raw_uid, uid in zip(unique_uids, uids)}
    ratings['uid'] = np.array([raw_uid2uid[raw_uid] for raw_uid in ratings.uid], dtype=int)
    tagging['uid'] = np.array([raw_uid2uid[raw_uid] for raw_uid in tagging.uid], dtype=int)

    # Reindex iid
    unique_iids = np.sort(movies.iid.unique()).astype(int)
    iids = np.arange(unique_iids.shape[0]).astype(int)
    raw_iid2iid = {raw_iid: iid for raw_iid, iid in zip(unique_iids, iids)}
    movies['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in movies.iid], dtype=int)
    ratings['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in ratings.iid], dtype=int)
    tagging['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in tagging.iid], dtype=int)

    # Create tid
    unique_tags = np.sort(tagging.tag.unique()).astype(str)
    tids = np.arange(unique_tags.shape[0]).astype(int)
    tags = pd.DataFrame({'tid': tids, 'tag': unique_tags})
    tag2tid = {tag: tid for tag, tid in zip(unique_tags, tids)}
    tagging['tid'] = np.array([tag2tid[tag] for tag in tagging.tag], dtype=int)
    tagging = tagging.drop(columns=['tag'])

    return movies, ratings, tagging, tags

num_feat_core = 10
num_core = 10

movies = movies.drop_duplicates()
ratings = ratings.drop_duplicates()
tagging = tagging.drop_duplicates()

movies = movies[movies.iid.isin(ratings.iid.unique())]
ratings = ratings[ratings.iid.isin(movies.iid.unique())]
tagging = tagging[tagging.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.uid.isin(ratings.uid.unique())]

movie_count = ratings['iid'].value_counts()
movie_count.name = 'movie_count'
ratings = ratings[ratings.join(movie_count, on='iid').movie_count > num_core]

user_count = ratings['uid'].value_counts()
user_count.name = 'user_count'
ratings = ratings[ratings.join(user_count, on='uid').user_count > num_core]

movies = movies[movies.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.uid.isin(ratings.uid.unique())]

tag_count = tagging['tag'].value_counts()
tag_count.name = 'tag_count'
tagging = tagging[tagging.join(tag_count, on='tag').tag_count > num_feat_core]

years = movies.year.to_numpy()
years[years < 1950] = 1950
movies['year'] = years

years = movies.year.to_numpy().astype(int)
min_year = min(years)
max_year = max(years)
num_years = (max_year - min_year) // 10
discretized_years = [min_year + i * 10 for i in range(num_years + 1)]
for i in range(len(discretized_years) - 1):
    years[(discretized_years[i] <= years) & (years < discretized_years[i + 1])] = str(
    discretized_years[i])
    years[years < discretized_years[0]] = discretized_years[0]
    years[years >= discretized_years[-1]] = discretized_years[-1]

movies['year'] = years

movies, ratings, tagging, tags = reindex_df_mlsmall(
                    movies, ratings, tagging)

In [39]:
ratings = ratings.sort_values('timestamp')
# ratings = ratings.sample(frac=1, random_state=42)

In [47]:
tf_no = 8

In [46]:
tf_size = len(ratings) // tf_no

user_hist = set()
movie_hist = set()

for i in range(tf_no):
    # print(i)
    tf = ratings[i*tf_size:(i+1)*tf_size]
    # print(len(tf['uid'].unique()))
    users = tf['uid'].unique()
    movies = tf['iid'].unique()
    new_users = set(users) - user_hist
    new_movies = set(movies) - movie_hist
    user_hist = user_hist.union(set(users))
    movie_hist = movie_hist.union(set(movies))
    # print timeframe, users, movies, new_users, new_movies in a nice format
    print(f'{i+1} ratings: {len(ratings)} users: {len(users)}, movies: {len(movies)}, new users: {len(new_users)}, new movies: {len(new_movies)}')


1 ratings: 79619 users: 51, movies: 275, new users: 51, new movies: 275
2 ratings: 79619 users: 43, movies: 341, new users: 39, new movies: 79
3 ratings: 79619 users: 54, movies: 676, new users: 48, new movies: 352
4 ratings: 79619 users: 29, movies: 856, new users: 27, new movies: 308
5 ratings: 79619 users: 21, movies: 1086, new users: 13, new movies: 159
6 ratings: 79619 users: 20, movies: 999, new users: 12, new movies: 40
7 ratings: 79619 users: 31, movies: 1013, new users: 19, new movies: 35
8 ratings: 79619 users: 39, movies: 1026, new users: 20, new movies: 113
9 ratings: 79619 users: 25, movies: 1083, new users: 15, new movies: 68
10 ratings: 79619 users: 29, movies: 1156, new users: 17, new movies: 88
11 ratings: 79619 users: 27, movies: 1106, new users: 13, new movies: 91
12 ratings: 79619 users: 28, movies: 1141, new users: 15, new movies: 24
13 ratings: 79619 users: 39, movies: 1160, new users: 18, new movies: 67
14 ratings: 79619 users: 29, movies: 971, new users: 13, new

In [44]:
users

array([598,  61, 509, 102, 339, 303, 517, 489, 412,  49, 210, 327, 203,
       378, 207, 231, 565, 152, 316, 247,  67, 208, 415, 583, 336, 360,
       270,  97, 294, 460, 512,  17, 554, 246, 256,  76,  24, 593, 182,
       329])

In [45]:
user_hist

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
tf = ratings[i*tf_size:(i+1)*tf_size]

In [ ]:
tf['uid'].unique()

array([600, 601, 602, 603, 604, 605, 606, 607])

In [ ]:
counts = ratings['uid'].value_counts().to_numpy()
print(f'cv: {np.mean(counts)/np.std(counts)}')

In [ ]:
len(ratings['uid'].unique())

In [ ]:
ratings.groupby("uid").count().iid.values